<center>
<img src="https://upload.wikimedia.org/wikipedia/commons/a/a8/%D0%9B%D0%9E%D0%93%D0%9E_%D0%A8%D0%90%D0%94.png" width=400/>
    <b style="font-size: 30px">Python. Итераторы и генераторы</b>
    <br/>
    <br/>
<b style="font-size: 20px">Вадим Мазаев</b>
</center>

# Итераторы

In [ ]:
for i in range(5):
    pass
    
for line in open('file.txt'):
    pass

for key in {'A' : 1, 'B' : 2, 'C' : 3}:
    pass
    
for letter in 'Hello, World':
    pass

In [1]:
iterable = [1, 2, 3]
iterator = iterable.__iter__()
iterator

In [2]:
iterator.__next__()

1

In [3]:
iterator.__next__()

2

In [4]:
iterator.__next__()

3

In [5]:
iterator.__next__()

StopIteration: 

### Iterable

Это объект у которого определён метод `__iter__`,

возвращающий **итератор**.

Примеры: `list`, `dict`, `range`

### Iterator
Это объект у которого определён метод `__next__`.

Метод `__next__` при каждом вызове возвращает следующий элемент 

последовательности или выкидывает исключение  `StopIteration`.

## iter & next

In [6]:
iterator = iter([1])
iterator

In [7]:
next(iterator)

1

In [8]:
next(iterator)

StopIteration: 

In [9]:
next(iterator, 'some value')

'some value'

## Вторая форма оператора iter

In [1]:
import io
from functools import partial

stream = io.StringIO('abcdefghi')
read_3 = partial(stream.read, 3)

In [2]:
iter(read_3, '')

In [3]:
for chunk in iter(read_3, ''):  # iter(callable, sentinel)
    print(chunk, end=' ')

abc def ghi 

In [15]:
import typing as tp

def make_timer(ticks: int) -> tp.Callable[[], int]:
    
    def timer() -> int:
        nonlocal ticks
        ticks -= 1
        return ticks

    return timer

In [17]:
for i in iter(make_timer(10), -1):
    print(i, end=' ')

9 8 7 6 5 4 3 2 1 0 

## Реализация цикла for через while

In [ ]:
for value in sequence:
    ...

In [ ]:
iterator = iter(sequence)
while True:
    try:
        value = next(iterator)
    except StopIteration:
        break
    else:
        ...

## О хранении итератором состояния

In [18]:
iterable = range(10)
iterable

range(0, 10)

In [19]:
iterator = iter(iterable)
iterator

In [20]:
iterator is iter(iterator)

True

In [21]:
zip_iterator = zip(iterator, iterator)
zip_iterator

In [22]:
for pair in zip_iterator:
    print(pair, end=' ')

(0, 1) (2, 3) (4, 5) (6, 7) (8, 9) 

## Объект и его итератор

In [23]:
iterable = open('IteratorsGenerators.ipynb')
iterable

<_io.TextIOWrapper name='IteratorsGenerators.ipynb' mode='r' encoding='UTF-8'>

In [24]:
iter(iterable)

<_io.TextIOWrapper name='IteratorsGenerators.ipynb' mode='r' encoding='UTF-8'>

In [25]:
iterable is iter(iterable)

True

In [ ]:
class TextIOWrapper:
    def __iter__(self) -> 'TextIOWrapper':
        return self
    
    def __next__(self) -> tp.Any:
        # do smth with state
    
    ...

In [4]:
class FilelikeRange:
    def __init__(self, start: int, stop: int) -> None:
        self.stop = stop
        self.index = start

    def __iter__(self) -> 'FilelikeRange':
        return self

    def __next__(self) -> int:
        if self.index == self.stop:
            raise StopIteration()
        value = self.index
        self.index += 1
        return value

In [5]:
for i in FilelikeRange(0, 5):
    print(i, end=' ')

0 1 2 3 4 

In [28]:
iterable = [1, 2, 3, 4, 5]
iterable

[1, 2, 3, 4, 5]

In [29]:
iter(iterable)

In [30]:
iterable is iter(iterable)

False

In [6]:
class ListlikeRange:
    class Iterator:
        def __init__(self, start: int, stop: int) -> None:
            self.stop = stop
            self.index = start

        def __next__(self) -> int:
            if self.index == self.stop:
                raise StopIteration()
            value = self.index
            self.index += 1
            return value

    def __init__(self, start: int, stop: int) -> None:
        self.stop = stop
        self.start = start

    def __iter__(self) -> Iterator:
        return self.Iterator(self.start, self.stop)

In [7]:
for i in ListlikeRange(0, 5):
    print(i, end=' ') 

0 1 2 3 4 

## Исчерпаемость

In [8]:
filelike_range = FilelikeRange(1, 5)
listlike_range = ListlikeRange(1, 5)

In [9]:
for elem in filelike_range:
    print(elem, end=' ')

1 2 3 4 

In [10]:
for elem in filelike_range:
    print(elem, end=' ')

In [11]:
for elem in listlike_range:
    print(elem, end=' ')

1 2 3 4 

In [12]:
for elem in listlike_range:
    print(elem, end=' ')

1 2 3 4 

## Sequence (упрощенный протокол)

In [14]:
import typing as tp
T = tp.TypeVar('T')

class Sequence:
    def __init__(self, *args: T) -> None:
        self.args = args

    def __getitem__(self, index: int) -> T:
        if index < 0 or index >= len(self.args):
            raise IndexError(index)  # expected by for to detect eos
        return self.args[index]

In [15]:
seq = Sequence(1, 2, 3, 4, 5)
seq[0], seq[2], seq[4]

(1, 3, 5)

In [16]:
for i in seq:
    print(i, end=' ')

1 2 3 4 5 

## \_\_contains__

In [45]:
3 in range(5)

True

In [46]:
# https://docs.python.org/3.9/reference/expressions.html#membership-test-details
# default __contains__ looks like
def __contains__(self, value: tp.Any) -> bool:
    for item in self:
        if item is value or item == value:
            return True
    return False

In [47]:
class MyRange:
    def __contains__(self, value: int) -> bool:
        return 0 <= value < self.stop
    
    ...

In [48]:
seq = Sequence(2, 3, 5, 8, 13, 21)

In [49]:
for i in seq:
    print(i, end=' ')

2 3 5 8 13 21 

In [50]:
8 in seq  # object has no __contains__, so "in" uses iteration over __getitem__

True

# Генераторы

In [17]:
import typing as tp

def countdown(n: int) -> tp.Iterator[int]:
    print(f'Counting down from {n}')
    for i in range(n, 0, -1):
        yield i
    print('Done')

In [18]:
for i in countdown(5):
    print(i)

Counting down from 5
5
4
3
2
1
Done


In [19]:
countdown

<function __main__.countdown(n: int) -> Iterator[int]>

In [20]:
counter = countdown(10)
counter

<generator object countdown at 0x7f0f7c335ac0>

In [12]:
iter(counter) is counter

True

In [8]:
counter = countdown(2)

In [9]:
next(counter)

Counting down from 2


2

In [10]:
next(counter)

1

In [11]:
next(counter)

Done


StopIteration: 

**Генератор** – это специальный итератор, который получается

в результате вызова функции, содержащей ключевое слово `yield`.

Последовательность значений, которую возвращает генератор,

задается последовательностью операторов `yield` в теле функции.

In [13]:
import inspect

In [15]:
def foo() -> None:
    yield

In [16]:
def bar() -> None:
    pass

In [17]:
inspect.isgeneratorfunction(foo)

True

In [18]:
inspect.isgenerator(foo())

True

In [19]:
inspect.isgeneratorfunction(bar)

False

### Примеры

In [20]:
def squares(size: int) -> int:
    for i in range(size):
        yield i ** 2

In [21]:
generator = squares(5)

In [22]:
for elem in generator:
    print(elem, end=' ')

0 1 4 9 16 

In [23]:
for elem in generator:
    print(elem, end=' ')

Генераторы истощаются!

In [24]:
T = tp.TypeVar('T')

def unique_ordered(elements: tp.Iterable[T]) -> tp.Iterator[T]:
    seen = set()
    for elem in elements:
        if elem in seen:
            continue
        seen.add(elem)
        yield elem

In [25]:
for elem in unique_ordered([1, 2, 3, 1, 2, 4]):
    print(elem, end=' ')

1 2 3 4 

### Цепочка генераторов

In [33]:
def sum_of_squares_of_even(iterable: tp.Iterable[int]) -> int:
    sum_ = 0
    for i in iterable:
        if i % 2 != 0:
            continue
        sum_ += i ** 2
    return sum_

In [34]:
sum_of_squares_of_even(range(10))

120

In [35]:
def even(iterable: tp.Iterable[int]) -> list[int]:
    result = []
    for i in iterable:
        if i % 2 != 0:
            continue
        result.append(i)
    return result

In [36]:
def squares(iterable: tp.Iterable[int]) -> list[int]:
    result = []
    for i in iterable:
        result.append(i ** 2)
    return result

In [37]:
sum(squares(even(range(10))))

120

In [38]:
def even(iterable: tp.Iterable[int]) -> tp.Iterator[int]:
    for elem in iterable:
        if elem % 2 == 0:
            yield elem

In [39]:
def squares(iterable: tp.Iterable[int]) -> tp.Iterator[int]:
    for elem in iterable:
        yield elem ** 2

In [40]:
sum(squares(even(range(10))))

120

Цепочка генераторов позволяет легко декомпозировать алгоритм

без существенных затрат памяти.

### Генераторные выражения

In [41]:
squares = (x ** 2 for x in range(5))
squares

<generator object <genexpr> at 0x7f1bb9a47510>

In [42]:
for square in squares:
    print(square, end=' ')

0 1 4 9 16 

In [1]:
max(x for x in range(10_000_000_000) if x % 11 == 0)

9999999999

In [ ]:
max([x for x in range(10_000_000_000) if x % 11 == 0])  # ~20G RAM

In [4]:
import sys

int_size_bytes = sys.getsizeof(0)
int_count = 10_000_000_000 / 11
list_size_bytes = int_size_bytes * int_count
list_size_gigabytes = list_size_bytes / (1024 ** 3)
list_size_gigabytes

20.319765264337715

### Генераторы в качестве итераторов

In [5]:
import typing as tp
from dataclasses import dataclass

@dataclass
class BinaryTreeNode:
    value: int
    left: tp.Optional['BinaryTreeNode'] = None
    right: tp.Optional['BinaryTreeNode'] = None

    def __iter__(self) -> tp.Iterator[int]:  # in-order
        for value in (self.left or ()):
            yield value

        yield self.value

        for value in (self.right or ()):
            yield value

In [6]:
tree = BinaryTreeNode(
    left=BinaryTreeNode(
        left=BinaryTreeNode(value=1),
        value=2,
    ),
    value=3,
    right=BinaryTreeNode(
        value=4,
        right=BinaryTreeNode(value=5),
    ),
)

In [7]:
for value in tree:
    print(value, end=' ')

1 2 3 4 5 

### enumerate

In [8]:
for i, char in enumerate('sample'):  # enumerate(iterable, start=index)
    print(i, char)

0 s
1 a
2 m
3 p
4 l
5 e


### zip

In [9]:
for left, right in zip('ABCD', 'xy'):
    print(left + right)

Ax
By


In [10]:
x = [1, 2, 3]
y = [4, 5, 6]
zipped = zip(x, y)
for item in zipped:
    print(item, end=' ')

(1, 4) (2, 5) (3, 6) 

In [11]:
zipped = zip(x, y)
x2, y2 = zip(*zipped)
print(x2, y2)

(1, 2, 3) (4, 5, 6)


### map & filter

In [12]:
for squared in map(lambda x: x ** 2, range(5)):
    print(squared, end=' ')

0 1 4 9 16 

In [13]:
for filtered in filter(lambda x: x % 2 == 0, range(10)):
    print(filtered, end=' ')

0 2 4 6 8 

### itertools.chain

In [14]:
from itertools import chain

In [15]:
for elem in chain(range(5), [10, 20], 'sample', [[i] for i in range(5)]):
    print(elem, end=' ')

0 1 2 3 4 10 20 s a m p l e [0] [1] [2] [3] [4] 

In [41]:
def repeat(times: int, obj: tp.Any) -> tp.Iterator[int]:
    for _ in range(times):
        yield obj

In [42]:
list(repeat(5, [1, 2, 3]))

[[1, 2, 3], [1, 2, 3], [1, 2, 3], [1, 2, 3], [1, 2, 3]]

In [43]:
for elem in chain.from_iterable(repeat(5, [1, 2, 3])):
    print(elem, end=' ')

1 2 3 1 2 3 1 2 3 1 2 3 1 2 3 

### Как проверить объект на итерируемость (быстро)

In [ ]:
try:
    iter(object_to_test)
except TypeError:
    # not an iterable
    ...
else:
    # iterable
    ...

### itertools.islice

In [18]:
from itertools import islice

In [19]:
sequence = range(10)

In [20]:
list(islice(sequence, 2, 5))  # seq[2:5]

[2, 3, 4]

In [21]:
sequence = range(10)

In [22]:
list(islice(sequence, 1, 7, 2))  # seq[1:7:2]

[1, 3, 5]

### itertools.count, cycle, repeat

In [23]:
from itertools import count, cycle, repeat

In [26]:
list(islice(count(start=0, step=3), 10))

[0, 3, 6, 9, 12, 15, 18, 21, 24, 27]

In [24]:
list(islice(cycle('sample'), 10))

['s', 'a', 'm', 'p', 'l', 'e', 's', 'a', 'm', 'p']

In [25]:
list(islice(repeat(42), 10))

[42, 42, 42, 42, 42, 42, 42, 42, 42, 42]

### itertools.dropwhile, takewhile

In [27]:
from itertools import dropwhile, takewhile

In [28]:
list(takewhile(lambda x : x < 5, range(10)))

[0, 1, 2, 3, 4]

In [29]:
list(dropwhile(lambda x : x < 5, range(10)))

[5, 6, 7, 8, 9]

### itertools.tee

In [30]:
from itertools import tee

![tee](https://4.bp.blogspot.com/-u_KYBwIUyF4/UUR5cvbv6PI/AAAAAAAAAXs/hPJT0ZR5iBc/s1600/tee_diagram.png)

In [44]:
iterable1, iterable2 = tee(range(3), 2)

In [32]:
for elem in iterable1:
    print(elem, end=' ')

for elem in iterable2:
    print(elem, end=' ')

0 1 2 0 1 2 

### itertools: combinatorics

In [33]:
from itertools import product, permutations, combinations

In [34]:
list(product('AB', 'XY'))  # reads in-memory + shallow copy

[('A', 'X'), ('A', 'Y'), ('B', 'X'), ('B', 'Y')]

In [35]:
list(permutations('ABC'))

[('A', 'B', 'C'),
 ('A', 'C', 'B'),
 ('B', 'A', 'C'),
 ('B', 'C', 'A'),
 ('C', 'A', 'B'),
 ('C', 'B', 'A')]

In [36]:
list(combinations('ABC', 2))

[('A', 'B'), ('A', 'C'), ('B', 'C')]

### itertools.groupby

In [37]:
from itertools import groupby

In [38]:
for key, group in groupby('AAAABBBCCD'): 
    print(key, list(group))

A ['A', 'A', 'A', 'A']
B ['B', 'B', 'B']
C ['C', 'C']
D ['D']


In [48]:
words = ['cab', 'face', 'cafe', 'abc', 'goo']
words = sorted(words, key=sorted)
words

['cab', 'abc', 'face', 'cafe', 'goo']

In [55]:
for key, group in groupby(words, key=sorted): 
    print(','.join(key), list(group))

a,b,c ['cab', 'abc']
a,c,e,f ['face', 'cafe']
g,o,o ['goo']


<img src="https://i.imgur.com/mM6OdDr.png" alter="agents-of-yield" width=900/>

#### Image source: https://speakerdeck.com/hollodotme/marvelous-agents-of-yield

## Генераторы: продвинутое использование
#### Inspired by: http://dabeaz.com/finalgenerator/

In [100]:
def create_generator() -> tp.Iterator[int]:
    yield 5

In [101]:
def create_generator() -> tp.Generator[int, None, None]:
    yield 5

In [96]:
def create_duplicator() -> tp.Generator[int, int, None]:
    print('Give me a value, please')
    value = yield
    print(f'Got value: {value}')
    yield value * 2
    print('Finished')

In [97]:
dublicator = create_duplicator()
next(dublicator)

Give me a value, please


In [98]:
dublicator.send(21)

Got value: 21


42

In [99]:
dublicator.send(100500)

Finished


StopIteration: 

### yield как выражение

![yield-expr](https://i0.wp.com/storage.googleapis.com/ssivart/super9-blog/priming-generator.png?w=1200&ssl=1)

In [60]:
def jumping_counter(upto: int) -> tp.Generator[int, int, None]:
    count = 1
    while count <= upto:
        jump = yield count
        count += jump or 1

In [61]:
generator = jumping_counter(3)

In [62]:
next(generator)  # equals to .send(None)

1

In [63]:
generator.send(2)

3

In [64]:
next(generator)

StopIteration: 

### throw

In [65]:
generator = jumping_counter(5)

In [66]:
next(generator)

1

In [67]:
generator.throw(Exception, 'Good luck!')

Exception: Good luck!

### close

In [70]:
generator = jumping_counter(5)

In [71]:
next(generator)

1

In [72]:
generator.close()

In [73]:
next(generator)

StopIteration: 

### Обработка close

In [74]:
def create_generator() -> tp.Iterator[int]:
    while True:
        try:
            yield 42
        except GeneratorExit:  # can't be ignored
            print('Exiting...')
            return

In [75]:
generator = create_generator()

In [76]:
next(generator)

42

In [77]:
generator.close()

Exiting...


### @contextmanager

In [78]:
from contextlib import contextmanager
import tempfile
import shutil

In [79]:
# https://github.com/python/cpython/blob/b3f0ceae919c1627094ff628c87184684a5cedd6/Lib/contextlib.py#L142
@contextmanager
def tempdir():
    dirname = tempfile.mkdtemp()
    try:
        yield dirname
    finally:
        shutil.rmtree(dirname)

In [80]:
with tempdir() as path:
    print(path)

/tmp/tmpttmjlu__


### yield from

In [81]:
T = tp.TypeVar('T')

def repeat(times: int, iterable: tp.Iterable[T]) -> T:
    for _ in range(times):
        yield from iterable  # https://www.python.org/dev/peps/pep-0380/

In [82]:
for elem in repeat(5, [1, 2, 3]):
    print(elem, end=' ')

1 2 3 1 2 3 1 2 3 1 2 3 1 2 3 

In [83]:
def repeat(times: int, iterable: T) -> T:
    for _ in range(times):
        yield iterable

In [84]:
for elem in repeat(5, [1, 2, 3]):
    print(elem, end=' ')

[1, 2, 3] [1, 2, 3] [1, 2, 3] [1, 2, 3] [1, 2, 3] 

### yield & return

In [104]:
def create_generator() -> tp.Generator[int, None, int]:
    yield 42
    return 21

In [105]:
generator = create_generator()
next(generator)
next(generator)

StopIteration: 21

In [106]:
def generator_wrapper():
    result = yield from create_generator()
    print(result)

In [107]:
list(generator_wrapper())

21


[42]

## Генераторы в качестве корутин

### Многозадачность

![multitasking](https://miro.medium.com/max/1580/1*r94wLYporfXxgaIakEfBIA.png)

А что, если использовать генератор в качестве корутины?

У генератора уже есть необходимый нам интерфейс:

In [108]:
def pinger() -> tp.Generator[None, None, None]:
    while True:
        print('ping')
        yield

In [109]:
def ponger() -> tp.Generator[None, None, None]:
    while True:
        print('pong')
        yield

Давайте напишем простенький планировщик задач (scheduler):

In [110]:
from collections import deque
from time import sleep

In [ ]:
def run(*tasks: tp.Generator[None, None, None]) -> None:
    try:
        # code here...
    except KeyboardInterrupt:
        return

In [91]:
def run(*tasks: tp.Generator[None, None, None]) -> None:
    task_queue = deque(tasks)
    try:
        while task_queue:
            sleep(0.5)
            task = task_queue.popleft()
            try:
                task.send(None)
            except StopIteration:
                continue
            task_queue.append(task)
    except KeyboardInterrupt:
        return

In [93]:
run(pinger(), ponger())

ping
pong
ping
pong
ping
pong
ping
pong
ping
pong


Кроме того, эту идею можно развить, и возвращать из корутины

специальный объект, описывающий действие, которое должен совершить

планировщик.

Например, создать новую задачу:

In [ ]:
def spawner() -> tp.Generator[Task, tp.Any, None]:
    print('Spawn new task')
    task_id = yield NewTask(pinger())
    for _ in range(5):
        print('tick')
        yield
    yield KillTask(task_id)
    print('Task killed')

Именно это вам и нужно будет реализовать в домашней задаче `pyos`

# Спасибо за внимание!